# RAG Workshop - Naive RAG Challenges

This notebook demonstrates the key limitations of naive RAG systems using our extended Wikipedia dataset. We'll focus on scenarios that clearly show where naive RAG fails and why advanced techniques are necessary.

## Dataset Overview:

- **61 articles** including Wikipedia + long technical blogs from Lilian Weng, arXiv papers
- **1,210 pre-chunked** pieces with 300 character chunks, 50 character overlap
- **Pre-embedded** using OpenAI text-embedding-3-small
- **Cloud-hosted** on Qdrant for reliable access
- **Includes cross-domain articles** to demonstrate naive RAG limitations

# 1. Setup Your Own Qdrant Cloud Collection

For this workshop, you'll need to set up your own Qdrant Cloud instance and ingest the extended Wikipedia dataset. Follow these steps:

## 🚀 Prerequisites Setup

### Step 1: Create Qdrant Cloud Account
1. Go to [Qdrant Cloud](https://cloud.qdrant.io/)
2. Sign up for a free account
3. Create a new cluster (free tier is sufficient)
4. Get your cluster URL and API key from the dashboard

### Step 2: Set Environment Variables
Create a `.env` file in the project root with:
```bash
OPENAI_API_KEY=your_openai_api_key_here
QDRANT_URL=your_qdrant_cluster_url_here
QDRANT_API_KEY=your_qdrant_api_key_here
```

### Step 3: Run Data Ingestion Script
Before running this notebook, execute the ingestion script to populate your Qdrant collection:

```bash
# From the project root directory
python scripts/ingest_to_qdrant_cloud.py
```

This script will:
- ✅ Load the extended Wikipedia dataset (61 articles)
- 🔪 Create 1,210 chunks with 300 character chunks, 50 character overlap  
- 🤖 Generate embeddings using OpenAI text-embedding-3-small
- 📤 Upload everything to your Qdrant Cloud collection
- ⏱️ Takes approximately 5-10 minutes to complete

**⚠️ Important**: Make sure you have run the ingestion script successfully before proceeding with this notebook!

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

## 1.1. Connect to Your Qdrant Cloud Collection

Now that you've run the ingestion script, let's connect to your Qdrant collection and verify the data is loaded correctly.

In [ ]:
import os
from openai import OpenAI
from qdrant_client import QdrantClient

# Verify environment variables are set
required_vars = ["OPENAI_API_KEY", "QDRANT_URL", "QDRANT_API_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print("❌ Missing required environment variables:")
    for var in missing_vars:
        print(f"   - {var}")
    print("\n💡 Please set these in your .env file and restart the notebook")
    raise ValueError("Environment variables not configured")

# Initialize OpenAI client
openai_client = OpenAI()

# Initialize Qdrant Cloud client
qdrant_client = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY")
)

# Collection configuration
collection_name = "workshop_wikipedia_extended"
embedding_model = "text-embedding-3-small"

print(f"✅ Connected to Qdrant Cloud")
print(f"📚 Collection: {collection_name}")
print(f"🤖 Embedding model: {embedding_model}")
print(f"🌐 Qdrant URL: {os.getenv('QDRANT_URL')}")

✅ Connected to Qdrant Cloud
📚 Collection: workshop_wikipedia_extended
🤖 Embedding model: text-embedding-3-small


## 1.2. Verify Collection and Dataset

Let's verify that your ingestion was successful and the data is properly loaded:

In [ ]:
try:
    # Get collection information
    collection_info = qdrant_client.get_collection(collection_name)
    point_count = collection_info.points_count
    
    if point_count == 0:
        print("⚠️ Collection exists but is empty!")
        print("💡 Please run the ingestion script: python scripts/ingest_to_qdrant_cloud.py")
    else:
        print(f"📊 Collection Statistics:")
        print(f"   Total chunks: {point_count:,}")
        print(f"   Vector dimension: {collection_info.config.params.vectors.size}")
        print(f"   Distance metric: {collection_info.config.params.vectors.distance}")
        
        if point_count == 1210:
            print("✅ Expected number of chunks found! Ingestion was successful.")
        else:
            print(f"⚠️ Expected 1,210 chunks but found {point_count}. Ingestion may be incomplete.")

        # Sample a few points to see the data structure
        sample_points = qdrant_client.scroll(
            collection_name=collection_name,
            limit=3,
            with_payload=True,
            with_vectors=False
        )[0]

        print(f"\n📝 Sample data structure:")
        for i, point in enumerate(sample_points):
            payload = point.payload
            print(f"\nChunk {i+1}:")
            print(f"   Title: {payload.get('title', 'Unknown')}")
            print(f"   Text preview: {payload.get('text', '')[:100]}...")
            print(f"   Chunk {payload.get('chunk_index', 0)+1} of {payload.get('total_chunks', 0)}")
            
except Exception as e:
    print(f"❌ Error accessing collection '{collection_name}': {e}")
    print("\n💡 Troubleshooting:")
    print("1. Make sure you've run: python scripts/ingest_to_qdrant_cloud.py")
    print("2. Check your QDRANT_URL and QDRANT_API_KEY in .env file")
    print("3. Verify your Qdrant Cloud cluster is running")
    raise

📊 Collection Statistics:
   Total chunks: 1,210
   Vector dimension: 1536
   Distance metric: Cosine

📝 Sample data structure:

Chunk 1:
   Title: BERT (language model)
   Text preview: Bidirectional encoder representations from transformers (BERT) is a language model introduced in Oct...
   Chunk 1 of 10

Chunk 2:
   Title: BERT (language model)
   Text preview: Euclidean space. Encoder: a stack of Transformer blocks with self-attention, but without causal mask...
   Chunk 2 of 10

Chunk 3:
   Title: BERT (language model)
   Text preview: consists of a sinusoidal function that takes the position in the sequence as input. Segment type: Us...
   Chunk 3 of 10


## 2. Build the Q/A Chatbot

Now we can focus on the core RAG functionality without worrying about data preparation!

![../imgs/naive-rag.png](../imgs/naive-rag.png)

### 2.1. Retrieval - Search the cloud database for relevant embeddings

In [4]:
def vector_search(query, top_k=2):
    """Search the Qdrant Cloud collection for relevant chunks."""
    # Create embedding of the query
    response = openai_client.embeddings.create(
        input=query,
        model=embedding_model
    )
    query_embeddings = response.data[0].embedding
    
    # Similarity search using the embedding
    search_result = qdrant_client.query_points(
        collection_name=collection_name,
        query=query_embeddings,
        with_payload=True,
        limit=top_k,
    ).points
    
    return [result.payload for result in search_result]

### 2.2. Generation - Use retrieved chunks to generate answers

In [5]:
import json

def model_generate(prompt, model="gpt-4o"):
    """Generate response using OpenAI's chat completion."""
    messages = [{"role": "user", "content": prompt}]
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,  # Deterministic output
    )
    return response.choices[0].message.content

def prompt_template(question, context):
    """Create a prompt template for RAG."""
    return f"""You are an AI Assistant that provides answers to questions based on the following context. 
Make sure to only use the context to answer the question. Keep the wording very close to the context.

Context:
```
{json.dumps(context)}
```

User question: {question}

Answer in markdown:"""

def generate_answer(question):
    """Complete RAG pipeline: retrieve and generate."""
    # Retrieval: search the knowledge base
    search_result = vector_search(question)
    if not search_result:
        return "No relevant information found."
        
    
    # Generation: create prompt and generate answer
    prompt = prompt_template(question, search_result)
    return model_generate(prompt)

### 2.3. Test Basic RAG Functionality

In [6]:
# Test with a clear, unambiguous question first
question = "What does the word 'deep' in 'deep learning' refer to?"
search_result = vector_search(question, top_k=3)

print(f"🔍 Question: {question}")
print(f"\n📚 Retrieved Sources:")

# Generate answer
answer = generate_answer(question)
print(f"\n🤖 Generated Answer:")
print(answer)

🔍 Question: What does the word 'deep' in 'deep learning' refer to?

📚 Retrieved Sources:

🤖 Generated Answer:
The word "deep" in "deep learning" refers to the number of layers through which the data is transformed. More precisely, deep learning systems have a substantial credit assignment path (CAP) depth. The CAP is the chain of transformations from input to output, describing potentially causal connections between input and output. For a feedforward neural network, the depth of the CAPs is that of the network and is the number of hidden layers plus one (as the output layer is also parameterized). For recurrent neural networks, in which a signal may propagate through a layer more than once, the CAP depth is potentially unlimited.


## 4. RAG Evaluation with RAGAS

Now let's evaluate our naive RAG system using **RAGAS** to establish baseline performance metrics and quantify the confusion we've observed.

### Context-Focused Metrics:

1. **Context Precision**: How well are relevant chunks ranked at the top?
2. **Context Recall**: How much of the necessary information was retrieved?
3. **Context Relevancy**: How relevant is the retrieved context to the question?

We're using **RAGAS** because it's purpose-built for RAG evaluation and provides deep insights into context quality - the most critical component of RAG performance.

In [7]:
# Import the RAGAS evaluation utility
from rag_evaluator_v2 import evaluate_naive_rag_v2

# Run evaluation on the current RAG system using RAGAS
print("🔍 Evaluating your Naive RAG system with RAGAS...")
print("This will evaluate context quality metrics on 15 questions...\n")

baseline_results = evaluate_naive_rag_v2(
    vector_search_func=vector_search,
    generate_answer_func=generate_answer
)

🔍 Evaluating your Naive RAG system with RAGAS...
This will evaluate context quality metrics on 15 questions...

✅ Loaded 14 questions from evaluation dataset

Evaluating 14 questions...

Question 1/14: Who introduced the ReLU (rectified linear unit) ac...
Question 2/14: What was the first working deep learning algorithm...
Question 3/14: Which CNN achieved superhuman performance in a vis...
Question 4/14: When was BERT introduced and by which organization...
Question 5/14: What are the two model sizes BERT was originally i...
Question 6/14: What percentage of tokens are randomly selected fo...
Question 7/14: Who introduced the term 'deep learning' to the mac...
Question 8/14: Which three researchers were awarded the 2018 Turi...
Question 9/14: When was the first GPT introduced and by which org...
Question 10/14: What were the three parameter sizes of the first v...
Question 11/14: What is the 'one in ten rule' in regression analys...
Question 12/14: What is the essence of overfitting a

Evaluating:   0%|          | 0/14 [00:00<?, ?it/s]


RAGAS EVALUATION RESULTS

📋 INDIVIDUAL QUESTION SCORES:
------------------------------------------------------------
 1. 🟢 1.000 - Who introduced the ReLU (rectified linear unit) activation f...
 2. 🟢 1.000 - What was the first working deep learning algorithm and who p...
 3. 🟢 1.000 - Which CNN achieved superhuman performance in a visual patter...
 4. 🔴 0.000 - When was BERT introduced and by which organization?
 5. 🟢 1.000 - What are the two model sizes BERT was originally implemented...
 6. 🟢 1.000 - What percentage of tokens are randomly selected for the mask...
 7. 🔴 0.000 - Who introduced the term 'deep learning' to the machine learn...
 8. 🔴 0.000 - Which three researchers were awarded the 2018 Turing Award f...
 9. 🟢 1.000 - When was the first GPT introduced and by which organization?
10. 🟢 1.000 - What were the three parameter sizes of the first versions of...
11. 🟢 1.000 - What is the 'one in ten rule' in regression analysis?
12. 🟢 1.000 - What is the essence of overfitting 